In [ ]:
import bisect

age_borders = 2, 4, 13, 20, 65
age_descriptions = ('Baby', 'Toddler', 'Kid', 
                    'Teenager', 'Adult', 'Old Geezer')

age = float(input('And the age is ? '))

age_index = bisect.bisect_right(age_borders, age)
print(f'This is a {age_descriptions[age_index]}')

In [ ]:
def hello(func):
    def inner():
        print('Hello')
        return func()
    return inner

def goodbye(func):
    def inner():
        result = func()
        print('Goodbye')
        return result
    return inner

@hello
@goodbye
def a():
    print('I am "a"')

@hello
@goodbye
def b():
    print('I am "b"')

In [ ]:
AFFIRMATION = frozenset({'y', 'yes', 'yeh', 'k', 'si', 'ok', 'yeah', 'yup'})
REJECTION = frozenset({'n', 'no', 'nope', 'nah', 'exit', 'quit', 'bye'})

def yesno(msg: str = "Yes or No? ") -> bool :
    while True:  # valid response loop
        response = input(msg)
        if response:
            if response.lower() in AFFIRMATION:
                return True
            elif response.lower() in REJECTION:
                return False
        print('Please answer Yes or No.')

In [ ]:
print(yesno('Testing: '))

In [ ]:
with open('/Users/stuartmoore/nums.txt') as nums:
    total = 0  # running total of valid inputs
    count = 0  # number of valid inputs
    for line_no, line in enumerate(nums, start=1):
        try:
            total += float(line.strip())
            count += 1
        except ValueError as e:
            print(f'line {line_no} has bad data: {line}')
if count > 0:
    print(f'Average is {total/count:.2f}')

# Game: rock paper scissors Spoke lizard
This generates the rules from a doc string. Very compact approach.

In [ ]:
#! /usr/bin/env python3                                                                                                
# rock paper scissors Spock lizard game, rules follow:
'''                                                                                                                    
scissors cuts paper                                                                                                    
paper covers rock                                                                                                      
rock crushes lizard                                                                                                    
lizard poisons Spock                                                                                                   
Spock smashes scissors                                                                                                 
scissors decapitates lizard                                                                                            
lizard eats paper                                                                                                      
paper disproves Spock                                                                                                  
Spock vaporizes rock                                                                                                   
rock crushes scissors                                                                                                  
'''

from random import choice
import readline

RULES = list(map(str.split, __doc__.lower().strip().split('\n')))

OPTIONS = ({winner for winner, verb, loser in RULES}
           | {loser for winner, verb, loser in RULES})

PROMPT = f"Make your choice from: {', '.join(sorted(OPTIONS))} \n " \
            f"(or press return alone to exit)\n" \
            f" choice: "
                                     
def check(playera, playerb, rules=RULES):
    for rule in rules:
        winner, verb, loser = rule
        if (playera, playerb) == (winner, loser):
            return playera, ' '.join(rule)
        if (playerb, playera) == (winner, loser):
            return playerb, ' '.join(rule)

print('\n\nWelcome to the game of rock paper scissors Spock lizard\n\nThe rules are:\n')
print(__doc__)
print()
while True:
    while True:
        player = input(PROMPT).lower()
        if not player or player in OPTIONS:
            break
    if not player:
        break
    computer = choice(list(OPTIONS))    
    try:
        winner, rule = check(player, computer)
        result = 'You WIN!' if player == winner else 'You Lose!'
    except TypeError as e:
        result, rule = "TIED", 'matched'
    print(f'{player} v. {computer} -> {result} \t{rule}')
    print()

# Understanding numbers written as words

In [ ]:
UNITS = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
        ]
TENS = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]
SCALES = ["hundred", "thousand", "million", "billion", "trillion"]
number_words = {"and": (1, 0)}
for position, word in enumerate(UNITS):
    number_words[word] = (1, position)
for position, word in enumerate(TENS):
    number_words[word] = (1, position * 10)
for position, word in enumerate(SCALES):
    number_words[word] = (10 ** (position * 3 or 2), 0)

POSITION_WORDS = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
SUFFIXES = [('ieth', 'y'), ('th', '')]

def words_to_num(words):
    words = words.replace('-', ' ')
    words = words.replace(',', ' ')
    current = result = 0
    for word in words.strip().split():
        if word in POSITION_WORDS:
            scale, increment = 1, POSITION_WORDS[word]
        else:
            for ending, replacement in SUFFIXES:
                if word.endswith(ending):
                    word = f"{word[:-len(ending)]}{replacement}"
            if word not in number_words:
                raise ValueError(f"Illegal word: {word}")
            scale, increment = number_words[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0
    return result + current

# Temperature conversions

In [ ]:
''' temperature conversion '''

def tempconv_F_to_C(temp):
    return (temp - 32) / 1.8

def tempconv_C_to_F(temp):
    return (temp * 1.8) + 32

def tempconv_Ré_to_C(temp):
    return temp * 5/4

def tempconv_C_to_Ré(temp):
    return temp * 4/5

def tempconv_Ré_to_F(temp):
    return tempconv_C_to_F(tempconv_Ré_to_C(temp))

def tempconv_F_to_Ré(temp):
    return tempconv_C_to_Ré(tempconv_F_to_C(temp))

# build dictionary of temperature conversion functions, 
# (dropping from the keys the prefix of tempconv_ and removing the _ before and after to)
prefix = 'tempconv_'
prefix_len = len(prefix)
tempconv_funcs = {name[prefix_len:].replace('_to_', ' to '): func for name, func in globals().items() 
                 if callable(func) and name.startswith(prefix)
                }
# define set of temperature units available
UNITS = set([unit for temp in [k.partition(' to ') 
                for k in tempconv_funcs.keys()] 
                for unit in temp if unit != ' to '])
ORIG = 'C'  # default original temp
NEW = 'F'  # default to convert to

def temp_convert(temp, orig, new):
    if not(isinstance(temp, int) or isinstance(temp, float)):
        raise ValueError('Valid temperature not provided')
    if not orig or not new:
        raise ValueError('Original and/or New units not specified')
    func = tempconv_funcs.get(f'{orig} to {new}')
    if not func:
        raise TypeError(f'No coversion available for {orig} to {new}')
    return func(temp)

print('Welcome to this temperature conversion programme\n')
print(f'The following units are accepted: {", ".join(UNITS)}')
print('and the following conversions:-', end='')
print('', *tempconv_funcs.keys(), sep='\n * ')

while True:  # do another conversion loop
    while True:  # get valid unit selections from user 
        orig = input(f'What units to convert from? [{ORIG}]  ')
        if not orig:
            orig = ORIG
        new = input(f'What units to convert to? [{NEW if orig != NEW else ORIG}] ')
        if not new:
            new = NEW if orig != NEW else ORIG
        if orig != new and orig in UNITS and new in UNITS:
            break
        print(f'You need to select from the following units: {", ".join(UNITS)}')

    while True:  # get a numeric input for temperature
        try:
            temp = float(input(f'Temperature ({orig}): '))
            break
        except:
            print('Numeric value required')

    try:
        print(f'{temp:.2f} degrees {orig} is {temp_convert(temp, orig, new):.2f} degrees {new}')
    except ValueError as e:
        print(e)
    except TypeError as e:
        print(e)

    if input('Another? [Yes/no] ').lower() not in ('y', 'yes', '1', 'yup', 'ok', ''):
        break

# validated numeric input (int or float)

In [ ]:
    def get_num(msg):
        'prompt user with msg and return valud numerical input, int or float'
        while True:    
            response = input('Some prompt: ')
            num = None
            try:
                num = int(response)
                break
            except ValueError as e:
                try:
                    num = float(response)
                    break
                except ValueError:
                    pass
            print('Please enter a number')
        return num

    num = get_num('Please enter a number: ')
    print(f"You entered: {num} of type {type(num)}")

# Crude way to check for a prime number

In [ ]:
def is_prime(num):
    if num == 1: return False
    if num in (2, 3): return True
    if not num % 2: return False
    for value in range(3, int(num**0.5+1), 2):
        if not num % value: return False
    return True

# Determing day of week for any date (no libraries)


![formulae](https://cs.uwaterloo.ca/~alopez-o/math-faq/img297.gif)

k is day (1 to 31)

m is month (1 = March, ..., 10 = December, 11 = Jan, 12 = Feb) Treat Jan & Feb as months of the preceding year

C is century (1987 has C = 19)

Y is year (1987 has Y = 87 except Y = 86 for Jan & Feb)

W is week day (0 = Sunday, ..., 6 = Saturday)

From: https://cs.uwaterloo.ca/~alopez-o/math-faq/node73.html

In [ ]:
''' determine day of week for any Gregorian date '''

# based on formulae at https://cs.uwaterloo.ca/~alopez-o/math-faq/node73.html

DOW = dict(enumerate(('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday')))
MONTHS = dict(enumerate(('January', 'February', 'March', 'April',
                       'May', 'June', 'July', 'August',
                       'September', 'October', 'November', 'December'), start=1))



def is_leap_year(year):
    'return True/False if year is a leap year'
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def get_date(msg):
    'get date from user and return tuple of dd, mm, yyyy'
    while True:
        response = input(msg).strip().split('/')
        if response and len(response) == 3:
            try:
                dd, mm, yyyy = [int(x) for x in response]
                if not 1 <= dd <= 31:
                    raise IndexError('Day must be between 1 and 31')
                if not 1 <= mm <= 12:
                    raise IndexError('Month must be between 1 and 12')
                if not 1582 <= yyyy <= 9999:
                    raise IndexError('Year must be between 1582 and 9999')
                if mm == 2 and dd > 28:
                    if not is_leap_year(yyyy) or dd > 29:
                        raise IndexError(f'Too many days for {MONTHS[2]} in {yyyy}')
                if dd == 31 and mm not in (1, 3, 5, 7, 8, 10, 12):
                    raise IndexError(f'Too many days for {MONTHS[mm]}')
                return dd, mm, yyyy
            except ValueError as e:
                print(f'{response} can not be understood as a date such as 18/05/1958')
            except IndexError as e:
                print(e)
        else:
            print('A response in the required format is expected. Please try again.')

def day_of_week(dd, mm, yyyy):
    'return day of week for date dd/mm/yyyy'
    mm = mm - 2  # march = 1, jan = 11
    if mm < 1:
        mm + 12
    if mm >= 11:  # year - 1 if month is jan or feb
        yyyy -= 1

    C = int(yyyy / 100)
    Y = yyyy % (C * 100)
    W = (dd + int(2.6 * mm - 0.2) - 2 * C + Y + int(Y/4) + int(C/4)) % 7

    return DOW[W]

dd, mm, yyyy = get_date('Please enter date of birth in form: dd/mm/yyyy: ')
print(f'{dd}/{mm}/{yyyy} is a {day_of_week(dd, mm, yyyy)}')

# example of using itertools islice to chunk something up

In [ ]:
from itertools import islice

def outcomes():
    'check scores are in range, return list of results'

    def chunk(it, size):
        'split a sequence into tuple chunks of same size'
        it = iter(it)
        return iter(lambda: tuple(islice(it, size)), ())

    Final = []
    name = ['James', 'Rod', 'Kate', 'Ann', 'Nick']
    count = [1,3,6,-2,7,8,11,-2,10,4,3,1,5,-999,9]
    results = {name_: counts_ for name_, counts_ in zip(name, list(chunk(count, 3)))}
    print(results)

    for name_, counts in results.items():
        if all( -1 < count_ < 11 for count_ in counts ):
            Final.append(f'{name_} passed.\n')
        elif -999 in counts:
            Final.append(f'{name_} had an error.\n')
        else:
            Final.append(f'{name_} failed.\n')

    return Final

print(''.join(outcomes()))

# function to merge strings (dropping any overlapping text)

In [ ]:
def str_merge(left: str, right: str) -> str:
    'return merged string, without duplication in overlap'
    chars = 0  # number of overlapping characters
    for ind in range(1, min(len(right), len(left)) + 1):
        print(left[-ind:], right[:ind], left.endswith(right[:ind]))  # DEBUG line
        if left.endswith(right[:ind]):
            chars = ind
    return left + right[chars:]
        
# usage example:        
print('result: ', str_merge('abc', 'cfghiabc'))

# nest list sorting using itemgetter from operator

In [ ]:
from operator import itemgetter

data = [['name3', 0.3, 10], ['name1', 0.2453022035139001, 5], ['name2', 0.2116333563690437, 15]]
print(data)
data.sort(key=itemgetter(1))  # i.e. sort on second element 
print(data)

# using bisect to find a dictionary boundary entry

In [ ]:
import bisect

RATES = {0: 0.0005,
         100000: 0.0010, 
         200000: 0.0015, 
         250000: 0.0025, 
         500000: 0.0040, 
         1000000: 0.0055, 
         2000000: 0.0060, 
         5000000: 0.0070}

RATES_ = list(RATES.keys())

def findInterestEarned2(p, m, t):

    def calc_for_rate():
        if p in RATES_:
            rate_index = RATES_.index(p)
        else:
            rate_index = bisect.bisect(RATES_, p) - 1
        rate = RATES[RATES_[rate_index]]
        calc = (m * 12) + ((p + ( m * 12 )) * rate )
        interest = ( (p + (m * 12)) * rate )
        return calc, interest

    counter = 0
    interest = 0

    if p >= 0 and m >= 0 and t > counter:
        for counter in range(t):
            calc, interest_ = calc_for_rate()
            interest += interest_
            p += calc
    return (interest)

print(findInterestEarned2(200000, 12, 12))

In [ ]:
def file ():
    letter_groups = ["abc", "def", "ghi", "jkl", "mno", "pqrs", "tuv", "wxyz"]
    result=[]
    with open("first.txt","rt") as first:
        for line in first:
            for character in line:
                for idx, seq in enumerate(letter_groups, start = 2):
                    if character in seq:
                        result.append(idx)
                        break
        return "".join([str(idx) for idx in result])

In [ ]:
print(file())

In [ ]:
def file2 ():
    letter_groups = {"abc": 2, "def":3 , "ghi": 4, "jkl": 5, "mno": 6, "pqrs": 7, "tuv": 8, "wxyz": 9}
    result=[]
    with open("first.txt","rt") as first:
        for line in first:
            for character in line:
                seq = [seq for seq in letter_groups.keys() if character in seq]
                if seq:
                    result.append(letter_groups[seq[0]])
        return "".join([str(idx) for idx in result])

In [ ]:
print(file2())

In [ ]:
def indexofmin(list1):
    'return index position of lowest value in list (first occurence)'
    min_idx = 0  # assume first entry is lowest
    lowest = list1[0]  # using variable to save looking up list1[index] every loop
    idx = 1  # start comparisons from element 1
    for value in list1[1:]:
        if value < lowest:
            min_idx = idx
            lowest = value
        idx += 1
    return min_idx

x = input("Enter a list of space separated numbers: ")
try:
    list1 = [float(num) for num in x.split()]
except ValueError as e:
    print('invalid number in list')
else:
    result = indexofmin(list1)
    print(result)

In [ ]:
def chunks(array, size):
    'Yield successive sized chunks from array'
    for idx in range(0, len(array), size):
        yield array[idx:idx+size]

Array = [[1,2], [3, 4], [5, 6], [7, 8]]
new_array =[element for element in chunks(arr, 2)]
a = new_array[0]
b = new_array[1]

In [ ]:
d = [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114]
c = [201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214]
h = [301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314]
s = [401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414]
deck = [d, c, h, s]

firstColor = int(input("Color of the first card: (1 = Diamonds, 2 = Clubs, 3 = Hearts, 4 = Spades)\n"))
firstValue = int(input("Value of the first card: (1 = A, 11 = J, 12 = Q, 13 = K)\n"))
secondColor = int(input("Color of the second card: (1 = Diamonds, 2 = Clubs, 3 = Hearts, 4 = Spades)\n"))
secondValue = int(input("Value of the second card: (1 = A, 11 = J, 12 = Q, 13 = K)\n"))

dP = []
cP = []
hP = []
sP = []
playerDeck = [dP, cP, hP, sP]

def firstCard(i):
    switcher = {
        1: dP.insert(0, 100 + firstValue),
        2: cP.insert(0, 200 + firstValue),
        3: hP.insert(0, 300 + firstValue),
        4: sP.insert(0, 400 + firstValue)
        }
    return switcher.get(i)

firstCard(firstColor)
firstCard(secondColor)

print(playerDeck)

In [ ]:
firstCard(4)

In [ ]:
dP

In [ ]:
entry = 'dummy'
scores = []
while entry:  # keep looping until entry is empty
    entry = input('Enter next score: ')
    if entry:  # if entry is not empty
        try:
            score = int(entry)  # convert string to an intger, if fails except will catch it
            scores.append(score)
        except ValueError as e:
            print('Sorry, do not understand that score')

total = sum(scores)
number_of_scores = len(scores)
average = total / number_of_scores

print(f'Total: {total}, average: {average:.2f} of {number_of_scores} scores')

In [ ]:
def move(my_history, their_history, my_score, their_score):
    ''' Arguments accepted: my_history, their_history are strings.
    my_score, their_score are ints.
    
    Make my move.
    Returns 'c' or 'b'. 
    '''

    # my_history: a string with one letter (c or b) per round that has been played with this opponent.
    # their_history: a string of the same length as history, possibly empty. 
    # The first round between these two players is my_history[0] and their_history[0].
    # The most recent round is my_history[-1] and their_history[-1].
    
    # Analyze my_history and their_history and/or my_score and their_score.
    # Decide whether to return 'c' or 'b'.
    print(f'moving - {my_history}, {their_history}, {my_score}, {their_score}')
    if len(my_history)==0:
        return 'c'
    if their_history[:-3]=='bbc': #If last 3 has 2 Betrays, Betray
        return 'b'
    if their_history[:-3]=='cbb':
        return 'b'
    if their_history[:-3]=='bcb':
        return 'b'
    if their_history[:-3]=='ccb': #If last 3 has 2 Colludes, collude
        return 'c'
    if their_history[:-3]=='cbc':
        return 'c'
    if their_history[:-3]=='bcc':
        return 'c'
    if their_history[:-3]=='bbb': #If last 3 has 3 Betrays, BETRRAY
        return 'b'
    if their_history[:-3]=='ccc': #If last 3 has 3 colludes, collude x3 >.<
        return 'c'

if True:
    
    #Test Move function
    print(move('', '', 0, 0))
    print(move('bcbcbc','bcbcbc',10,-10))
    print(move('bcbcbcbcbcbbcbbcbbbcbcbccbcbcccccc', 'bcbcbcbcbcbbcbbcbbbcbcbccbcbcccccc', -30, -10))
    print(move('ccccc', 'ccccc', 0, 0))
    print(move('bbbbb', 'bbbbb', 0, 0))
    print("All tests passed")

In [ ]:
    import time
    import random

    print(
    """American Military History Database, version 0.0.3.1
    Coded by Heath
    -
    Please note that this program is a work in progress and new articles, topics, and other
    info is being added every day.
    -
    -
    -
    Search any weapon, battle, vehicle, or person involved in any of the United
    State's past or present involvement in any amred conflicts below.
    -
    """)

    # primary dictionary with prefered capitalisation
    WW2_keywords = {
        "WW2": ["WW2", "ww2", "World War 2", "world war 2", "World War Two", "world war two"],
        "WW2 Keywords": ["World War 2 Keywords", "world war 2 keywords", "World War 2 keywords", "World War Two Keywords", "world war two keywords", "World War Two keywords", "WW2 Keywords", "ww2 keywords", "WW2 keywords", "ww2 Keywords"],
        "WW2 Weapons": ["WW2 weapons", "ww2 weapons", "WW2 Weapons"],
        "WW2 Vehicles": ["WW2 vehciles", "ww2 vehicles", "WW2 Vehicles"],
        "WW2 Battles": ["WW2 battles", "ww2 battles", "WW2 Battles"],
        "WW2 Fronts": ["WW2 fronts", "ww2 fronts", "WW2 Fronts"],
        "WW2 Theatres": ["WW2 theatres", "ww2 theatres", "WW2 Theatres"],
        "WW2 Axis Powers": ["Axis powers", "Axis Powers", "the Axis Powers", "The Axis Powers", "The axis powers", "the axis powers"],
        "WW2 Allied Powers": ["Allied powers", "Allied Powers", "the Allied Powers", "The Allied Powers", "The allied powers", "the allied powers"],
        "WW2 Major Events ": ["WW2 major events", "WW2 Major Events", "ww2 major events", "WW2 Major events", "ww2 Major Events"],
        "WW2 Leaders": ["WW2 leaders","ww2 leaders", "WW2 Leaders", "ww2 Leaders"],
        "WW2 Officers": ["WW2 famous officers", "ww2 famous officers", "ww2 Famous officers"]
        }

    # copy of above with everything lowercased, and the original key included for lookup on primary
    ww2_keywords_lower = {key.lower(): (key, [words.lower() for words in terms]) 
                          for key, terms in WW2_keywords.items()}

    while True:  # continue inviting user to search until they enter return on its own
        s = input("Search: (blank to exit) ").lower()
        if not s:  # no input so user does not want to search anymore
            break

        if s in ww2_keywords_lower['ww2'][1]:
            print("""(paragraph) For more information on World War Two, search 'WW2 Keywords'""")

        elif s in ww2_keywords_lower['ww2 keywords'][1]:
            print("The following are keywords regarding 'World War Two':")
            print(*WW2_keywords['WW2 Keywords'], sep='\n')    

        else:
            if s in ww2_keywords_lower:
                original_key = ww2_keywords_lower[s][0]  # find original primary key
                print(f"The following articles have been found on {original_key}: ")
                print(*WW2_keywords[original_key], sep='\n')          
            else:
                print("ERROR; Article does not exist, be more specific.")

In [ ]:
from random import randint

def dice_roll():
    return randint(1,12)

def getpot(prompt):
    while True:
        response = input(prompt)
        if not response:
            return None
        try:
            cash = int(response)
            if cash < 1:
                raise ValueError("Please enter a positive number for the bet")
            break
        except ValueError as e:
            print(e)

    return cash


print("welcome to the casino, how about a game of lucky sevens?\n")
print("the goal of the game is to roll the dice and roll a total of 7\n")                    

while True:
    pot = getpot("Please enter the amount of money you want to bet (return to exit)\n")
    if not pot:
        break

    maxpot = pot
    total_rolls = 0
    table = []

    while pot > 0 and total_rolls < 1000:
        print(f'Pot: ${pot}')
        dice_roll_total = dice_roll()
        win = dice_roll_total == 720
        table.append((win, pot))
        pot += 4 if win else -1
        if pot > maxpot:
            maxpot = pot
        total_rolls += 1
        print(f"{'Win' if win else 'Lose'} -> roll #{total_rolls:4}: {dice_roll_total:2}   ", end="")

    print(f'Final pot {pot}')
    print(f'Highest pot {maxpot}')
    print(f'Number of rolls {total_rolls}')
    print()

    # table
    print('Status | Roll #  | Pot')
    print('========================')
    for idx, (win, pot) in enumerate(table):
        print(f"{'Win ' if win else 'Lose'}     {idx + 1:4}     {pot:4}   ")

In [ ]:
list1=['a1','b1','c1']  
list2 = ['a2','b2', 'c2']

data1= 'c1'
data2= 'c2'
for d1, d2 in zip(list1, list2):
    if data1 == d1 and data2 == d2:
        print('yes')
        break
else:
    print('no')

In [ ]:
    trigger = 10  # each <trigger> words, insert <marker> before the word
    marker = '^'  # test to insert before the <trigger># word
    words_count = -1
    with open('random.txt') as words_file, open('new_randon.txt', 'w') as new_file:
        words_count = 0
        for line in words_file:
            words = line.split()
            words_num = len(words)
            new_line = ""
            for idx, word in enumerate(words, start=1):
                words_count += 1
                if words_count == trigger:
                    new_line += marker
                    words_count = 0
                new_line += word + (' ' if idx < words_num else '')
            new_file.write(new_line + '\n')    

In [ ]:
    num = []
    idx = 1
    while True:
        entry = input(f"Enter entry number {idx} (or just enter to finish): ")
        if not entry:
            break
        try:
            num.append(int(entry))
            idx += 1
        except ValueError as e:
            print('Entry must be an integer or (empty to exit)')

    print(*num, sep=', ')

In [ ]:
    supers = {'superman': 'Superman', 'batman': 'Batman'}

    while True:  # search loop
        query = input('Who do you want to search for? (just return to exit) ').lower()
        if not query:
            break  # exit from search loop
        if query in supers:
            search = supers[query]
            # do the search
            print('Searched for ' + search)
        else:
            print('Sorry, don\'t know about that super.')

In [ ]:
    name = input('Good Day, Stranger. What is your name? ')
    if name:
        while True:
            num_cats_str = input(f'How many cats do you have {name}? ')
            try:
                num_cats = int(num_cats_str)
            except ValueError:
                print('You did not enter a Number. Please enter something in the form of "7 or 2, etc"')
            else:
                if num_cats < 0:
                    print('Eh! Eh! Buckeroo! That horse poo, ain\'t gonna fly here. Now enter a number.')
                else:
                    break
        if num_cats >= 4:
            print('That\'s a lot of catteroos, son!')
        else:
            print('Ain\'t that many catteroos, hey!')

In [ ]:
    ''' prime number table generator and test using sieve method '''

    def prime_num_table(num):
        ''' Return list of flags 1 for prime, 0 for not prime indexed for range 0 to n '''

        def sieve(prime_table):
            ''' modify flags for all numbers that are not prime from 4 to sqrt(num) '''
            for check_num in range(2, int(num**(0.5)+1)):
                if prime_table[check_num]:
                    for set_multiple in range(check_num * 2, num + 1, check_num):  # set all multiples of check_num
                        prime_table[set_multiple] = 0
            return prime_table

        if isinstance(num, int) and num > 0:
            prime_table = [1] * (num + 1)  # pre-populate table (list) as all primes
            prime_table[0] = 0  # neither prime nor composite technically
            prime_table[1] = 0  # neither prime nor composite technically
            sieve(prime_table)
            return prime_table
        else:
            raise ValueError(f'{num} argument provided. Postive integer argument required.')

    def is_prime_num(num, prime_table = None):
        ''' return True/False accordingly is argument n is prime number or not
            using table generated with the sieve method if table not supplied '''
        if prime_table is None:
            print_table = []
        if isinstance(num, int) and num > 0:
            if isinstance(prime_table, list):
                if not prime_table:
                    prime_table = prime_num_table(num)
                return bool(prime_table[num])
            else:
                raise ValueError(f'Truth table of primes from 0 to {num} required.')
        else:
            raise ValueError(f'{num} argument provided. Postive integer argument required.')

    if __name__ == "__main__":
        test_data = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 
                47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107]
        prime_table = prime_num_table(max(test_data))
        for test in test_data:
            print(f"{test:>4d} is{'' if is_prime_num(test, prime_table) else 'not'} a prime number.")

In [ ]:
list_ = ['one', 'two', 'three']
set_ = {'one', 'two', 'three'}
dict_ = {1: 'one', 'II': 'two', 'Third': 'three'}
tuple_ = 'one', 'two', 'three'
str_ = 'one two three'
print(len(list_), len(set_), len(dict_), len(tuple_), len(str_))

In [ ]:
vowels = ['a', 'e', 'i', 'o', 'i', 'u']
print(vowels.index('i'))

# create list of subclass instances from list of parent class instances
Say a load of data is processed into a list of instances of some parent class Foo (which can have multiple attributes). Then, additional data is processed that a subclass, say Bar, of the parent class is designed to hold. It is therefore necessary to create new instances of the subclass that calls the parent __init__ with the same data as the original parent instances as well as setting the additional attributes.

In [ ]:
class Foo:  # parent class

    def __init__(self, x, w=101):
        self.x = x  # example attribute - could be loads
        self.w = w  # example

class Bar(Foo):  # subclass

    def __init__(self, other, **kwargs):  # random number of attributes
        for key, value in kwargs.items():
          setattr(self, key, value)
        # create a dictionary of all of the attributes of the original parent class data
        other_attributes = {entry: getattr(other, entry) for entry in dir(other) if not entry.startswith('__')}
        super().__init__(**other_attributes)  # intialise in new subclass instance the same atttributes

def bar_from_foo(foo_instance, **kwargs):
    '''clone the attributes of a Foo in a new Bar subclass instance and add new attributes'''
    return Bar(foo_instance, **kwargs)

foos = [Foo(10), Foo(20), Foo(30)]  # initial set of data, typically more complex than one integer
bars = []
for foo in foos:
    bars.append(bar_from_foo(foo, y=20, z=30))  # creating bars with additional attributes to original data

for foo in foos: print(vars(foo))
for bar in bars: print(vars(bar))

# writing a simple csv file

In [ ]:
import csv

words = {'this': 10, 'is': 5, 'an': 4, 'example': 15}

'writing headers and then rows explicitly'
with open('example.csv', 'w') as example:
    writer = csv.writer(example)
    writer.writerow(["words", "frequency"])
    for word, frequency in words.items():
        writer.writerow([word, frequency])

'using field names as well'
with open('example2.csv', 'w') as example:
    fieldnames = ["words", "frequency"]
    writer = csv.DictWriter(example, fieldnames=fieldnames)
    writer.writeheader()
    for word, frequency in words.items():
        writer.writerow({'words': word, 'frequency': frequency})

In [1]:
    import csv
    fieldnames = ["Date", "Time", "Time Zone", "Description",
                  "Currency","Gross","Fee","Net","Balance",
                  "Transaction ID","From Email Address","Name",
                  "Bank Name","Bank account","Postage and Packaging Amount",
                  "VAT","Invoice ID","Reference Txn ID",
                 ]
    with open('example3.csv', 'w') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

In [11]:
raw_headers = '"Date", "Time", "Time Zone", "Description", "Currency","Gross","Fee","Net","Balance", "Transaction ID","From Email Address","Name", "Bank Name","Bank account","Postage and Packaging Amount", "VAT","Invoice ID","Reference Txn ID"'
fieldnames = list(csv.reader(raw_headers.splitlines()))[0]
print(fieldnames)

['Date', ' "Time"', ' "Time Zone"', ' "Description"', ' "Currency"', 'Gross', 'Fee', 'Net', 'Balance', ' "Transaction ID"', 'From Email Address', 'Name', ' "Bank Name"', 'Bank account', 'Postage and Packaging Amount', ' "VAT"', 'Invoice ID', 'Reference Txn ID']


In [16]:
raw_headers = '"Date", "Time", "Time Zone", "Description", "Currency","Gross","Fee","Net","Balance", "Transaction ID","From Email Address","Name", "Bank Name","Bank account","Postage and Packaging Amount", "VAT","Invoice ID","Reference Txn ID"'
split_headers = raw_headers.splitlines()
print(split_headers)
fieldnames = list(csv.reader(split_headers))
print(fieldnames)

['"Date", "Time", "Time Zone", "Description", "Currency","Gross","Fee","Net","Balance", "Transaction ID","From Email Address","Name", "Bank Name","Bank account","Postage and Packaging Amount", "VAT","Invoice ID","Reference Txn ID"']
[['Date', ' "Time"', ' "Time Zone"', ' "Description"', ' "Currency"', 'Gross', 'Fee', 'Net', 'Balance', ' "Transaction ID"', 'From Email Address', 'Name', ' "Bank Name"', 'Bank account', 'Postage and Packaging Amount', ' "VAT"', 'Invoice ID', 'Reference Txn ID']]


In [ ]:
def reg():
    uname = input("Please enter a username: ")
    pword = input("Please enter a password: ")
    with open("UNPW.txt", "a") as f:
        f.write("\nUsername:"+uname)
        f.write("\nPassword:"+pword)

def login():
    uname = input("Please enter your username: ").lower()
    pword = input("Please enter your password: ")
    found = False
    with open("UNPW.txt", "r") as f:
        for line in f:
            if line.strip().startswith('Username:'):
                if line.strip().lower().endswith(uname):
                    line = next(f)
                    if line.strip().startswith('Password:'):
                        if line.strip().endswith(pword):
                            found = True
                    break
    return found

#reg()
#reg()
#reg()
security_passed = login()
print(security_passed)

In [ ]:
#!/usr/bin/env python3

"""This program plays a game of Rock, Paper, Scissors between two Players,
and reports both Player's scores each round."""

from random import choice
from itertools import cycle

moves = ['rock', 'paper', 'scissors']
hand = ['rock', 'paper', 'scissors']
EXITS = ['quit', 'bye', 'stop', 'end', 'finish', 'exit']
number_of_rounds = 10

class Player:
    '''The Player class is the parent class for all of the Players
    in this game'''
    
    player_count = 0
    
    def __init__(self, name=''):
        self.score = 0
        self.moves = []
        self.other_moves = []
        Player.player_count += 1
        self.player_number = Player.player_count
        self.name = name
        
    def move(self):
        self.moves.append('rock')
        return self.moves[-1]

    def learn(self, their_move):
        self.other_moves.append(their_move)


    @staticmethod
    def beats(one, two):
        return ((one == 'rock' and two == 'scissors') or
               (one == 'scissors' and two == 'paper') or
               (one == 'paper' and two == 'rock'))
    
    def winner(p1, p2):  # p1 is self
        if p1.beats(p1.moves[-1], p2.moves[-1]):
            return p1
        elif p1.beats(p2.moves[-1], p1.moves[-1]):
            return p2
        else:
            return None

class RandomPlayer(Player):
    '''Makes moves based on random choices'''
    
    def move(self):
        self.moves.append(choice(hand))
        return self.moves[-1]

class HumanPlayer(Player):
    ''' Makes moves based on human input'''
    
    def __init__(self, name=''):
        super().__init__()
        while not name:
                name = input(f'Name of player {self.player_number}? ')
        self.name = name
                 
    def move(self):
        while True:
            move = input(f"Player {self.player_number}, {self.name}, enter move or 'quit': ").lower()
            if move in EXITS:
                return None
            if move not in hand:
                print(f"That is not one of the options, {', '.join(hand)}. Please try again.")
            else:
                print()
                self.moves.append(move)
                return self.moves[-1]

class ReflectPlayer(RandomPlayer):
    ''' Makes moves based on what opponent played last time'''
    
    def move(self):
        if self.other_moves:  # if not first play
            self.moves.append(self.other_moves[-1])
        else:
            super().move()
        return self.moves[-1]

class CyclePlayer(RandomPlayer):
    ''' Makes moves based on what played last time'''
    
    def __init__(self):
        self.next_move = cycle(hand)
        super().__init__()
    
    def move(self):
        if self.moves:  # if not first play
            self.moves.append(next(self.next_move))
        else:
            super().move()
            while next(self.next_move) != self.moves[-1]: pass
        return self.moves[-1]
        
            
class Game:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        self.round = 0
        self.p1_desc = f"Player 1{(', ' + self.p1.name) if self.p1.name else ''}"
        self.p2_desc = f"Player 2{(', ' + self.p2.name) if self.p2.name else ''}"

    def play_round(self):
        ' play one round of rock, papers, scissors with two player instances '
        
        move1 = self.p1.move()
        if not move1:  # player wants to quit
            return True
        move2 = self.p2.move()
        if not move2:  # player wants to quit
            return True

        print(f"{self.p1_desc:^15} {self.p2_desc:^15}")
        print(f"{move1:^15} {move2:^15}")
        self.p1.learn(move2)
        self.p2.learn(move1)
        win = self.p1.winner(self.p2)
        if not win:
            print(f'{"this round is a draw":^31}')
        elif win is self.p1:
            print(f'{"wins":^15}')
            self.p1.score += 1
        else:
            print(f'{" ":15} {"wins":^15}')
            self.p2.score += 1
        return False

    def scores(self):
        ' print scores of players so far '
        score1 = 'score: ' + f'{self.p1.score}'
        score2 = 'score: ' + f'{self.p2.score}'
        print(f"{score1:^15} {score2:^15}")

    def fini(self):
        ' print overall winner or draw '
        print("\nGame over!")
        if self.p1.score > self.p2.score:
            print(f'{self.p1_desc} wins overall')
        elif self.p2.score > self.p1.score:
            print(f'{self.p2_desc} wins overall')
        else:
            print('A draw overall')

    def play_game(self):
        ' play <number_of_rounds> or rock, paper, scissors with two player instances '
        print("Game start!")
        last_round = False
        while self.round < number_of_rounds and not last_round:
            self.round += 1
            print(f"\nRound {self.round}:")
            last_round = self.play_round()
            self.scores()
        self.fini()
        

if __name__ == '__main__':
    STRATEGIES = {'standard': Player, 'random': RandomPlayer,
              'human': HumanPlayer, 'reflect': ReflectPlayer, 
              'cycle': CyclePlayer}
    while True:
        print(f"Player types are: {', '.join(STRATEGIES.keys())}")
        player_one = input('First player type: ').lower()
        player_two = input('Second player type: ').lower()
        if player_one in STRATEGIES and player_two in STRATEGIES:
            break
        print('Both players need to be selected from the available player types.')
    game = Game(STRATEGIES[player_one](), STRATEGIES[player_two]())
    game.play_game()

In [ ]:
from random import randint
from collections import defaultdict

def main():
    min = 1
    max = 6
    rolls = defaultdict(int)
    roll_again = 'yes'
    while roll_again == 'yes':
        print('Rolling the dice...')
        roll1 = randint(min, max)
        roll2 = randint(min, max)
        print(roll1, roll2)
        rolls[roll1] += 1
        rolls[roll2] += 1
        roll_again = input('Roll again? ')
    print(rolls)
main()

In [ ]:
    from random import randint
    '''def rogue():
         strength = (random.randint(1,9))
         int(strength)-1
         luck = (random.randint(1,9)
         int(luck)+4
    print('what class?)
    userClass=input()
    print({},userClass.strength)
    '''

    class Characters():

          def __init__(self, strength=None, luck=None):
              if not strength:
                  strength = randint(1, 5)
              self.strength = strength
              if not luck:
                  luck = randint(1,4)
              self.luck = luck

    class Rogue(Characters):

          def __init__(self):
              super().__init__(randint(1, 9), randint(1, 9))

    player_types = {'basic': Characters, 'rogue': Rogue}

    while True:
        userClass = input(f'What type of character ({", ".join(player_types.keys())})? ').lower()
        if userClass in player_types:
            break
        print('Do not recognise that type. Please try again.')

    player1 = player_types[userClass]()
    print(player1.strength)

In [ ]:
#If the program works correctly you should be able to give it a txt with a word on each line and it should give you a new txt with the words sorted alphabetically.
with open(input("Specify txt you wish to sort: "),"r") as sortfile, open("SortedFile.txt","w") as newfile:
    for line in sorted(sortfile.readlines()):
        newfile.write(line)

In [ ]:
!cat SortedFile.txt

In [ ]:
!rm SortedFile.txt

In [ ]:
from collections import defaultdict

rolls = defaultdict(int)



In [ ]:
    def f(x = []):  # remember, default is evaluated when first read, not when called
        print('x id', id(x))
        return x

    print('Setting a ...', end = '')
    a = f()  # as no arg, a now references the [] object created on definition of function f
    print('Setting b ...', end = '')
    b = f()  # same again, reference to the originally defined object is returned
    print(f'a is b? {a is b}')  # True because a and b refer to exactly the same object in memory
    a.append(3)  # appending to that default object
    print('Setting c ...', end = '')
    c = f()  # refers to same object as a and b because default reference is used 
    print(f'values ... a: {a}, b: {b}, c: {c}')  # should be 3 because 3 was appended to the list by a.append(3)
    print(f'ids ... a: {id(a)}, b: {id(b)}, c: {id(c)}')  # just to show they are all same object

In [ ]:
    Setting a ...x id 4472423880
    Setting b ...x id 4472423880
    a is b? True
    Setting c ...x id 4472423880
    values ... a: [3], b: [3], c: [3]
    ids ... a: 4472423880, b: 4472423880, c: 4472423880

In [ ]:
    from random import randint

    def intro(min, max):
        print(f'I am going to think of a number between {min} and {max}.')
        print('Try and guess the number in as few tries as possible.')
        print('Good luck!')
        print()

    def main():
        replay = 'y'
        min = 1
        max = 100
        intro(min, max)
        num_guesses = 0
        while replay.lower() in ['y', 'yes', 'ok']:
            number = randint(1, 100)
            while True:
                guess = int(input('Guess a number! '))
                num_guesses += 1
                if guess > number + 11:
                    print('That guess is too high!')
                elif guess in range(number-10, number-1):
                    print("You're getting warmer, but that's a little too low!")
                elif guess in range(number + 1, number + 10):
                    print("You're getting warmer, but that's a little too high!")
                elif guess < number - 10:
                    print("That guess is too low!")
                elif guess == number:
                    print('Well done!')
                    print('You guessed my number in',num_guesses,'guesses!')
                    break 
            replay = input('Would you like to play again? (y=yes):')
    main()

In [ ]:
with open('page.txt', 'r') as f:
    line_counter = 0
    simple_array = []
    for line in f:
      if line.startswith('X-DSPAM-Confidence:'):
        temp_num = line[19:].strip()
        simple_array.append(temp_num)
        line_counter += 1
        print (temp_num)

In [ ]:
    def main():

        total = float(input("Enter starting balance: "))
        choice = 'dummy'
        while True:
            print(f"You have a balance of ${total:.2f}.")
            choice = input("Enter d for Deposit, w for Withdrawal, r for Repeating payments,\
    or q to Quit: ").lower()
            if not choice or choice in ['q', 'quit', 'stop', 'exit', 'x']:
                break
            if choice == 'd':
                while True:
                    dep = float(input("Enter deposit amount: "))
                    if not dep:
                        break
                    if dep > 0:
                        total += dep
                        print(f"You deposited ${dep:.2f}.", end=' ')
                        break
                    print("Deposit amount must be positive.", end=' ')

            elif choice == 'w':
                wD = float(input("Enter withdrawal amount: "))
                if wD <= total and total > 0:
                    total -= wD
                    print(f'You withdrew ${wD:.2f},', end=' ')
                else:
                    print("You do not have sufficient funds. Withdrawal denied!")     

            elif choice == 'r':
                pay = float(input("Enter payment amount: "))
                num = int(input("Enter number of payments: "))
                if num > 0:
                    for counter in range(1, num + 1):
                        if total >= pay:
                            total -= pay
                            print(f"Payment {counter} for ${pay} made. Your balance is now ${total:.2f}")
                        else:
                            print('insufficient funds to make any more payments')
                            break
            else:
                print('Sorry, that option is not currently available.')

        print('Your balance is $', total, ". Have a nice day!", sep='')
    main()

In [ ]:
def maximum_frequency(new_list):
    word_counter = {}
    for word in new_list:
        if word in word_counter:
            word_counter[word] += 1
        else:
            word_counter[word] = 1
    freq_order = sorted(word_counter, key=word_counter.get, reverse=True)
    return freq_order, word_counter

words = []
with open('random.txt') as f:
    for line in f:
        words.extend(line.strip().split())

words_in_freq_order, words_freq_table = maximum_frequency(words)

highest_freq = words_freq_table[words_in_freq_order[0]]
for word in words_in_freq_order:
    freq = words_freq_table[word]
    if freq < highest_freq:
        break
    print(word, freq)

In [ ]:
    #import csv
    #import keahelp

    def print_menu():
        print(" Menu option 1 - Fancy Unicorn financial journal")
        print(" Menu Option 2 - Trial balance")
        print(" Menu Option 3 - Net balance")
        print(" Menu option 4 - Profit per måned")
        print(" Menu option 5 - Exit")

    def print_sub_menu3():
        print(" Menu option 1 - Sales account")
        print(" Menu Option 2 - Inventory")
        print(" Menu Option 3 - Cash at bank")
        print(" Menu option 4 - Expenses")
        print(" Menu option 5 - Trade Recieveables")
        print(" Menu option 6 - Purchases")
        print(" Menu option 7 - Purchases returned")
        print(" Menu option 8 - Exit")

    '''f = open("jona096iJournal.csv", "r")
    csvreader = csv.reader(f,delimiter=";")
    Journalliste = []
    b = open("trialb.csv", "r")
    csvreader1 = csv.reader(b,delimiter=";")
    trialb = []
    c = open("profit.csv", "r")
    csvreader2 = csv.reader(c,delimiter=";")
    profit = []
    d = open("ledger.csv", "r")
    csvreader3 = csv.reader(d,delimiter=";")
    ledger = []'''

    print('Velkommen!')

    print(''' Vælg menu muligheder
    ---------------------------------------------------
    ''')

    while True:
        print_menu()
        choice = input("Vælg fra [1-5]: ")

        if choice == "1":
            pass
        elif choice == "2":
            print("Debit eller credit account")

            while True:
                print_sub_menu3()
                choice1 = input("Vælg fra [1-8]: ")

                if choice1 == "1":
                    pass
                elif choice1 == "8":
                    break

        elif choice == "4":
            '''for row in csvreader2:
                profit.append(str(row))
            for line in profit:
                print (line)'''
            pass

        elif choice == "5":
            break

In [ ]:
    while True:  # outloop, for each set of people to check
        yes = 0
        count = 0
        while True:  # inner loop for responses from a set of people
            input1 = input("Do you like chocolate? (y/n/q(uit)) ").upper()
            if not input1 or input1 in ['Q', 'QUIT', 'EXIT', 'BYE', 'X']:
                break
            if input1 in ["Y", "YES"]:
                print("You said you liked chocolate. ")
                yes += 1
                count += 1
            elif input1 in ["N", "NO"]:
                print("You put no. ")
                count += 1
            else:
                print("You didn't put y/n. ")

        print(f"{yes} people like chocolate out of {count} people who answered y/n. ")

        if not input('Another set of people? (y/n) ').lower() in ['y', 'yes', 'ok']:
            break

In [ ]:
    while True:
        multi_str = input("Are you alone or do you actually have friends? - Input number of players\n")
        try:
            multi = int(multi_str)
        except ValueError as e:
            print('You didn\'t enter a number. Please input amount of players.')
            continue
        if multi == 1:
            print('Lol what a loser. Okay singleplayer it is.\n')
            break
        elif multi == 2 or multi == 3 or multi == 4 or multi == 5:  # in would be better
            print(f' A total crew of {multi} friends. You\'re so cool!\n')
            break
        else:
            print("Please input amount of players")

In [ ]:
import math
print("This program will estimate Pi.")
n = 1
pi = 0
while True:
    pi_inverse_1 = ((2*math.sqrt(2))/9801)
    pi_inverse_2 = ((math.factorial(4*n))/(math.factorial(n))**4)
    pi_inverse_3 = (1103+(26390*n))/(396**(4*n))
    pi_inverse_4 = pi_inverse_1 * pi_inverse_2 * pi_inverse_3
    pi += (pi_inverse_4) ** (-1)
    print(pi)
    n += 1
    if pi_inverse_4 < 1e-15:
        break

In [ ]:
    from math import factorial
    from math import pow

    def pi_formula():
        sum = 0
        n = 0
        i = (math.sqrt(8))/9801

        while True:
            tmp = i*(factorial(4*n)/pow(factorial(n),4))*((26390*n+1103)/pow(396,4*n))
            sum +=tmp

            if(abs(tmp) < 1e-15):
                break
            n += 1

        return(1/sum)

    print(pi_formula())

In [ ]:
    list_dict_test = {'key1': 'test1', 'key2':[{'key2.1': 'test2.1', 'key2.2':'test2.2', 'key2.3':'test2.3'}], 'key3': 'test3', 'key4.1':[{'2ndkey4.1.': 'test4.1.', '2ndkey4.2.':'test4.2.', '2ndkey4.3.':'test4.3.'}], 'key5.1':[{'3rdkey5.1.': 'test5.1.', '3rdkey5.2.':'test5.2.', '3rdkey5.3.':'test5.3.'}], 'totalCount': 243, 'status': 0}

    def test(my_file):
        for key, value in my_file.items():
            if isinstance(value, list):
                print(f'outer key: {key}')
                for key, value in value[0].items():
                    print(f'inner key: {key}, inner value: {value}')
            else:
                print(f'outer key: {key}, outer value: {value}')


    test(list_dict_test)

In [22]:
    import cmath

    def quadratic(a, b, c):
        '''return solutions for quadratic equation ax2 + bx + c = 0, where
           a, b and c are real numbers and a ≠ 0'''

        d = (b**2) - (4 * a * c)
        sqrt_d = cmath.sqrt(d)
        return (-b + sqrt_d) / ( 2 * a ), (-b - sqrt_d) / (2 * a)

    print(quadratic(5, 8, 4))  # output can contain float or imaginary numbers

((-0.8+0.4j), (-0.8-0.4j))


In [29]:
    from itertools import islice

    def chunk(it, size):
        'split a sequence into tuple chunks of same size'
        it = iter(it)
        return iter(lambda: tuple(islice(it, size)), ())

    sample = [1, "one", 2, "two", 3, "three"]
    sample_chunks = chunk(sample, 2)

    for num, name in sample_chunks:
        print(f'{num} --- {name}')

1 --- one
2 --- two
3 --- three


In [51]:
sample = [2, 8, 3, 9, 4, 16, 5, 25]
it = iter(sample)
x = iter(lambda: tuple(islice(it, 3)), ())
print(x)

In [55]:
next(x)

StopIteration: 

In [59]:
    import sys
    import pygame
    from settings import Settings
    from ship import Ship
    import game_functions as gf
    
    
    def run_game():
        #initialize game and create screen object.         
        pygame.init()
        ai_settings=Settings()
        screen=pygame.display.set_mode((ai_settings.screen_width,ai_settings.screen_height))
        pygame.display.set_caption("Alien Invasion")

        #make a ship
        ship=Ship(screen)

        #set the background color
        bg_color=(230,230,230)

        #start the main loop for the game
        while True:
            gf.check_events()
        #watch for keyboard and mouse events
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                sys.exit()
        #redraw the screen during each pass through the loop
        screen.fill(ai_settings.bg_color)
        ship.blitme()
        #make the most recently drawn screen visible
        pygame.display.flip()
        

    run_game()

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


ModuleNotFoundError: No module named 'settings'

In [64]:
    def introduction(question):
        while True:
            answer = input(question).lower()
            if answer in ['yes', 'y', 'ok']:
                response = 'Yeah!'
                break
            elif answer in ['no', 'n', 'nah']:
                response = 'Goodbye!'
                break
            else:
                print('Sorry. Do not understand.')
        return response

    print(introduction('Are you single? '))

Are you single? nah
Goodbye!


In [66]:
list_ = ['one', 'two', 'three']
for idx, num_name in enumerate(list_, start=1):
    print(idx, num_name)

1 one
2 two
3 three


In [72]:
    from random import randint
    from collections import defaultdict

    def roll(list_of_rolls, die_type=6):
        list_of_rolls.append(randint(1, die_type))

    rolls = []  # this will be a list of rolls
    num_of_dice = 5
    for _ in range(num_of_dice):
        roll(rolls)
    hand_count = defaultdict(int)
    for dice in rolls:
        hand_count[dice] += 1
    print(hand_count)

defaultdict(<class 'int'>, {2: 1, 1: 1, 3: 1, 6: 1, 5: 1})


In [76]:
print(*("*" * count for count in range(10)), sep="\n")


*
**
***
****
*****
******
*******
********
*********


In [80]:
    def index_responses(questions):
        number = []
        answer = []
        for num_count, question in enumerate(questions, start=1):
            number.append(f"Q{num_count:0>2}.")
            answer.append(question)  # these are questions, not answers - just sayin'
        z = dict(zip(number, answer))
        return z

    print('Testing index_responses ()')
    print(index_responses(['a', 'b', 'c']))
    print(index_responses(['a','a','c','b']))

    def index_responses(questions):
        numbered_questions = {}
        for num_count, question in enumerate(questions, start=1):
            numbered_questions[f"Q{num_count:0>2}."] = question
        return numbered_questions

    print('Testing index_responses ()')
    print(index_responses(['a', 'b', 'c']))
    print(index_responses(['a','a','c','b']))

    def index_responses(questions):
        return {f"Q{num_count:0>2}.": question 
                for num_count, question in enumerate(questions, start=1)
               }

    print('Testing index_responses ()')
    print(index_responses(['a', 'b', 'c']))
    print(index_responses(['a','a','c','b']))

Testing index_responses ()
{'Q01.': 'a', 'Q02.': 'b', 'Q03.': 'c'}
{'Q01.': 'a', 'Q02.': 'a', 'Q03.': 'c', 'Q04.': 'b'}
Testing index_responses ()
{'Q01.': 'a', 'Q02.': 'b', 'Q03.': 'c'}
{'Q01.': 'a', 'Q02.': 'a', 'Q03.': 'c', 'Q04.': 'b'}
Testing index_responses ()
{'Q01.': 'a', 'Q02.': 'b', 'Q03.': 'c'}
{'Q01.': 'a', 'Q02.': 'a', 'Q03.': 'c', 'Q04.': 'b'}


In [81]:
    fileinput = 'contest.txt'
    contestants = {}
    elim = 1

    with open(fileinput, 'r') as f:
        for line in f.readlines():
            name, *scores = line.strip('\n').split(' ')
            checkscore = [float(score) for score in scores]
            if min(checkscore) > elim:
                contestants[name] = scores

In [82]:
contestants

{'Audria': ['4.66', '6.87', '4.93', '2.13', '7.90', '5.51', '2.46', '5.82'],
 'Curtis': ['2.99', '5.11', '4.72', '2.10', '7.90', '1.27', '4.78', '6.07'],
 'Giuseppina': ['8.27',
  '1.18',
  '5.14',
  '7.24',
  '1.40',
  '1.66',
  '2.57',
  '1.89'],
 'Arlena': ['7.19', '2.17', '7.82', '3.41', '4.18', '8.09', '9.65', '7.01'],
 'Tangela': ['4.32', '9.21', '2.76', '9.43', '6.81', '10.8', '5.17', '9.40']}

In [87]:
    from itertools import combinations
    import sys

    def pair(items):
        return list(combinations(items, 2))

    students = []
    sys.argv = ["dummy", "students.txt", "2"]
    if len(sys.argv) < 3:
        sys.exit(f'Expected 2 arguments, {len(sys.argv) - 1} given')

    with open(sys.argv[1], 'r') as f:
        students = f.read().strip().splitlines()
        print(students)

    num_of_weeks = int(sys.argv[2])

    output = pair(students)
    print(output)

['Mary Smith', 'Helen Watts', 'Kevin Bing', 'Stephanie Google', 'Alta Vista', 'Yahoo Hooey', 'Apple V. Samsung']
[('Mary Smith', 'Helen Watts'), ('Mary Smith', 'Kevin Bing'), ('Mary Smith', 'Stephanie Google'), ('Mary Smith', 'Alta Vista'), ('Mary Smith', 'Yahoo Hooey'), ('Mary Smith', 'Apple V. Samsung'), ('Helen Watts', 'Kevin Bing'), ('Helen Watts', 'Stephanie Google'), ('Helen Watts', 'Alta Vista'), ('Helen Watts', 'Yahoo Hooey'), ('Helen Watts', 'Apple V. Samsung'), ('Kevin Bing', 'Stephanie Google'), ('Kevin Bing', 'Alta Vista'), ('Kevin Bing', 'Yahoo Hooey'), ('Kevin Bing', 'Apple V. Samsung'), ('Stephanie Google', 'Alta Vista'), ('Stephanie Google', 'Yahoo Hooey'), ('Stephanie Google', 'Apple V. Samsung'), ('Alta Vista', 'Yahoo Hooey'), ('Alta Vista', 'Apple V. Samsung'), ('Yahoo Hooey', 'Apple V. Samsung')]


In [88]:
import test

In [89]:
dir(test)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [90]:
import unittest

In [91]:
dir(unittest)

['BaseTestSuite',
 'FunctionTestCase',
 'SkipTest',
 'TestCase',
 'TestLoader',
 'TestProgram',
 'TestResult',
 'TestSuite',
 'TextTestResult',
 'TextTestRunner',
 '_TextTestResult',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__unittest',
 'case',
 'defaultTestLoader',
 'expectedFailure',
 'findTestCases',
 'getTestCaseNames',
 'installHandler',
 'load_tests',
 'loader',
 'main',
 'makeSuite',
 'registerResult',
 'removeHandler',
 'removeResult',
 'result',
 'runner',
 'signals',
 'skip',
 'skipIf',
 'skipUnless',
 'suite',
 'util']

In [92]:
    class Data:
        def __init__(self):
            self.value = 10
            self.type = int
        def __repr__(self):
            return f'I have a value of {self.value} of type {self.type}'

    a = Data()
    print(a)

I have a value of 10 of type <class 'int'>


In [101]:
    def comparator(x,y):
        return(x==y)

    def testing(tests):
        for idx, test in enumerate(tests, start=1):
            try:
                assert comparator(*test[0]) is test[1], "some error message"
            except AssertionError as e:
                print(f'Test {idx} {test[0]} FAILED, did not return {test[1]}\n{e}')
            else:
                print(f'Test {idx} {test[0]} PASSED (returned {test[1]})')

    test_data = [((-12,3), False), ((3,-12), False), ((-12,-12), True)]
    testing(test_data)

Test 1 (-12, 3) PASSED (returned False)
Test 2 (3, -12) PASSED (returned False)
Test 3 (-12, -12) PASSED (returned True)


In [106]:
    VOWELS = {'a', 'e', 'i', 'u'}

    def stutter(s, k):
        """(str) -> str
        Return a string of characters from s, but with each consonant
        repeated k times.
        >>> stutter('', 24)
        ''
        >>> stutter('aei', 5)
        'aei'
        >>> stutter('b', 2)
        'bb'
        >>> stutter('acrobat', 3)
        'acccrrrobbbattt'
        """
        def is_vowel(character):
            return character.lower() in VOWELS

        return ''.join([c if is_vowel(c) else c * k for c in s])


    print(stutter('', 24))
    print(stutter('aei', 5))
    print(stutter('b', 2))
    print(stutter('acrobat', 3))


aei
bb
acccrrrooobbbattt
